In [1]:
# ライブラリのインポート
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import random
from qulacs import QuantumState
from qulacs.state import inner_product
from qulacs import QuantumCircuit
from qulacs.gate import to_matrix_gate
from qulacs import QuantumState
from qulacs.gate import Identity, X,Y,Z #パウリ演算子
from qulacs.gate import H
from qulacs.gate import RX,RY,RZ #パウリ演算子についての回転演算

from qulacs.gate import BitFlipNoise

## 係数の絶対値の分布をプロットする関数

In [2]:
def show_distribution(state, nqubits):
    plt.bar([i for i in range(2, nqubits)], abs(state.get_vector()))
    plt.show()

nqubits = 5
a = []
b = []
sample = 10000
problist = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]

ここで、problistは・・・

In [3]:
state = QuantumState(nqubits)
state.set_zero_state()

def make_Hadamard(nqubits):
    Hadamard = QuantumCircuit(nqubits)
    for i in range(nqubits):
        Hadamard.add_gate(H(i))
    return Hadamard

def make_U_w(nqubits):
    U_w = QuantumCircuit(nqubits)
    CnZ = to_matrix_gate(Z(nqubits - 1))
    # i - th qubitが全て1の場合だけゲートを作用
    for i in range(nqubits - 1):
        control_index = i
        control_with_value = 1
        CnZ.add_control_qubit(control_index, control_with_value)
    U_w.add_gate(CnZ)
    return U_w

for i in problist:
    prob = i
    def make_U_s(nqubits):
        U_s = QuantumCircuit(nqubits)
        for i in range(nqubits):
            U_s.add_gate(H(i))

        # ノイズの準備
        index1 = 0
        index2 = prob
        #print(index1, index2
        Dphe_gate = DepphasingNoise(index1, index2)

        ## 内積を評価するために、解状態|1...1>を作っておく
        target_state = QuantumState(nqubits)
        target_state.set_computational_basis(2 ** nqubits - 1)

        ## グローバーのアルゴリズムの実行
        Hadamard = make_Hadamard(nqubits)
        U_w = make_U_w(nqubits)
        U_s = make_U_s(nqubits)

        result = []
        data2 = []
        # nqubits毎の反復回数
        ans = [1,1,2,3,4,6,8,12,17,25,35,50,71,100,142,201,284]
        c = ans[nqubits - 1]
        print(c)

        state = QuantumState(nqubits)
        state.set_zero_state()
        Hadamard.update_quantum_state(state)
        for j in range(10):
            state = QuantumState(nqubits)
            state.set_zero_state()
            Hadamard.update_quantum_state(state)
            result = []
            data1 = []
            for k in range(c):
                Dphe_gate.update_quantum_state(state)
                U_w.update_quantum_state(state)
                U_s.update_quantum_state(state)
                if k == c:
                    data1.append(np.linalg.norm(inner_product(state, target_state)))
            x = data2.append(np.linalg.norm(inner_product(state, target_state)))

        a.append(np.mean(data2))
        b.append(np.std(data2))
        print(prob)